In [ ]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import os.path
import os
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization
from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast
from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from numpy import *
import random
import sys
import csv
from skimage.transform import PiecewiseAffineTransform, warp
from skimage import data
%matplotlib inline

## 1. Set the model configuration parameters

In [ ]:
img_height = 300 # Height of the input images
img_width = 300 # Width of the input images

img_channels = 3 # Number of color channels of the input images

mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.

subtract_mean = [123, 117, 104] # The per-channel mean of the images in the dataset

swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we should set this to `True`, but weirdly the results are better without swapping.

# TODO: Set the number of classes.
n_classes = 1 

# The anchor box scaling factors used in the original SSD512
scales = [0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05]

# The anchor box aspect ratios used in the original SSD512; the order matters
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]]

two_boxes_for_ar1 = True

steps = [8, 16, 32, 64, 128, 256, 512] # The space between two adjacent anchor box center points for each predictor layer.

offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.

clip_boxes = False # Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries

variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are scaled as in the original implementation

normalize_coords = True

## 2. Load the model

In [ ]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = os.getcwd() + '/model/' + 'ssd512_radical_HSM.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'compute_loss': ssd_loss.compute_loss})

## 3. Load some images

Load some images for which you'd like the model to make predictions.

In [ ]:
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

images_dir_test = 'images\\'

# Generate CSV file for the images

labels_format = ['image_name','xmin','xmax','ymin','ymax','class_id']

csv_file_test = open('labels_test.csv', 'a', newline='')
writer_test = csv.DictWriter(csv_file_test, fieldnames=labels_format)
writer_test.writeheader()

images_raw_list = os.listdir(images_dir_test)

for file in images_raw_list:
    writer_test.writerow({'image_name': file, 'xmin': str(0), 'xmax': str(5), 'ymin': str(0), 'ymax': str(5), 'class_id': str(1)})
    
csv_file_test.close() 

In [ ]:
labels_filename_test = 'labels_test.csv'

val_dataset.parse_csv(images_dir = images_dir_test,
                       labels_filename = labels_filename_test,
                       input_format = labels_format,
                       include_classes = 'all',
                       random_sample = False,
                       ret = False,
                       verbose = True)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv10_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

batch_size = 16

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the test dataset:\t{:>6}".format(val_dataset_size))

## 4. Make predictions and save the output

In [ ]:
# Set the generator for the predictions.

predict_generator = val_dataset.generate(batch_size=1,
                                         shuffle=True,
                                         transformations=[convert_to_3_channels,
                                                          resize],
                                         label_encoder=None,
                                         returns={'processed_images',
                                                  'filenames',
                                                  'inverse_transform',
                                                  'original_images',
                                                  'original_labels'},
                                         keep_images_without_gt=False)

In [ ]:
# Generate samples.

for l in range(val_dataset_size):
    batch_images, batch_filenames, batch_inverse_transforms, batch_original_images, batch_original_labels = next(predict_generator)

    i = 0 # Which batch item to look at
    
    # Make predictions.

    y_pred = model.predict(batch_images)
    
    # Decode the raw predictions in `y_pred`.

    y_pred_decoded = decode_detections(y_pred,
                                       confidence_thresh=0.0001,
                                       iou_threshold=0.4,
                                       top_k=200,
                                       normalize_coords=normalize_coords,
                                       img_height=img_height,
                                       img_width=img_width)
    
     # Convert the predictions for the original image.

    y_pred_decoded_inv = apply_inverse_transforms(y_pred_decoded, batch_inverse_transforms)
    
    predicted = []
    length = len(y_pred_decoded_inv[i])
    
    if length is not 0:
        pred_list = y_pred_decoded_inv[i].tolist()

        if(length != 1):
            for kk in range(len(pred_list)):
                predicted.append(pred_list[kk])
        else:
            predicted = pred_list

        np.set_printoptions(precision=2, suppress=True, linewidth=90)
        # best predicted box:
        np.set_printoptions(precision=2, suppress=True, linewidth=90)

        sorted_y_pred_decoded_inv = sorted(predicted,key=lambda predicted:predicted[1])
        best_box = sorted_y_pred_decoded_inv.pop()
        
        # Draw the predicted boxes onto the image

        # Set the colors for the bounding boxes
        colors = plt.cm.hsv(np.linspace(0, 1, n_classes+1)).tolist()
        
        # TODO: set the classes names (dont remove 'background')
        classes = ['background', 'HSM']

        plt.figure(figsize=(20,12))
        plt.imshow(batch_original_images[i])

        current_axis = plt.gca()

        xmin = best_box[2]
        ymin = best_box[3]
        xmax = best_box[4]
        ymax = best_box[5]
        color = colors[int(best_box[0])]
        label = '{}: {:.2f}'.format(classes[int(best_box[0])], best_box[1])
        current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='red', fill=False, linewidth=2))  
        current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})
        
        # save the image with the predicted box
        plt.savefig('output\\' + str(l+1) + '.jpg')